[Original author](https://designwith.ml/assets/docs/Meetup_clustering_sample)  
  
Clustering meetup.com data! Using the location and group information to cluster the members into 6 clusters - but it is up to you to figure out what they mean!  

Data was on Kaggle.

In [1]:
#these are all of the libraries i'll be using - and I load the groups.csv data
import pandas as pd
import numpy as np
import random
import sklearn
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import matplotlib
%matplotlib inline
matplotlib.style.use('ggplot')

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [6]:
import pandas as pd

df = pd.read_csv(r'C:\Users\cpop4\OneDrive - Universitatea Babeş-Bolyai\Desktop\MACHINE LEARNING\ubb-sociology-ml\data\groups.csv')

# Data Step
This step we're going to look at our dataframes and become familiar with what's in them.

In [7]:
#this is what the groups.csv looks like as a dataframe - it is about the groups
df.head()

group_id  category_id          category.name     category.shortname  \
0      6388           14       health/wellbeing       health-wellbeing   
1      6510            4  community/environment  community-environment   
2      8458           26           pets/animals           pets-animals   
3      8940           29         sci-fi/fantasy         sci-fi-fantasy   
4     10104           26           pets/animals           pets-animals   

   city_id      city country              created  \
0    10001  New York      US  2002-11-21 16:50:46   
1    10001  New York      US  2003-05-20 14:48:54   
2    10001  New York      US  2004-03-27 09:55:41   
3    10001  New York      US  2002-11-16 04:49:16   
4    10001  New York      US  2003-10-22 21:39:49   

                                         description  \
0  Those who practice or hold a strong interest i...   
1  The New York Alternative Energy Meetupis for t...   
2                                          not_found   
3  Welcome to the The New York City Anime Meetup ...   
4  We welcome those who support pits, even if you...   

              group_photo.base_url  ...  \
0  https://secure.meetupstatic.com  ...   
1  https://secure.meetupstatic.com  ...   
2  https://secure.meetupstatic.com  ...   
3  https://secure.meetupstatic.com  ...   
4  https://secure.meetupstatic.com  ...   

                          organizer.photo.photo_link  \
0  https://secure.meetupstatic.com/photos/member/...   
1  https://secure.meetupstatic.com/photos/member/...   
2  https://secure.meetupstatic.com/photos/member/...   
3  https://secure.meetupstatic.com/photos/member/...   
4  https://secure.meetupstatic.com/photos/member/...   

                          organizer.photo.thumb_link organizer.photo.type  \
0  https://secure.meetupstatic.com/photos/member/...               member   
1  https://secure.meetupstatic.com/photos/member/...               member   
2  https://secure.meetupstatic.com/photos/member/...               member   
3  https://secure.meetupstatic.com/photos/member/...               member   
4  https://secure.meetupstatic.com/photos/member/...               member   

  rating state    timezone                    urlname utc_offset  \
0   4.39    NY  US/Eastern     alternative-health-nyc     -14400   
1   4.31    NY  US/Eastern  alternative-energy-meetup     -14400   
2   4.84    NY  US/Eastern                    Animals     -14400   
3   4.46    NY  US/Eastern                  NYC-Anime     -14400   
4   4.09    NY  US/Eastern                NYC-Pitbull     -14400   

       visibility                            who  
0          public            Explorers of Health  
1          public        Clean Energy Supporters  
2          public                  Animal Voices  
3          public                     Anime Fans  
4  public_limited  NYC Pits & People, Dog Lovers  

[5 rows x 36 columns]

In [16]:
#this is the df about the members
df2 = pd.read_csv(r'C:\Users\cpop4\OneDrive - Universitatea Babeş-Bolyai\Desktop\MACHINE LEARNING\ubb-sociology-ml\data\members.csv', encoding = "ISO-8859-1" , nrows=1600133) #  This encoding is commonly used for Western European languages and is a single-byte encoding that can represent a wide range of characters, including characters with accents and diacritics.

In [17]:
df2.shape

(1600133, 14)

In [18]:
df2.head()

member_id                                                bio      city  \
0          3                                          not_found  New York   
1          3                                          not_found  New York   
2          3  Hi, I'm Matt. I'm an entrepreneur who has star...  New York   
3          3  Hi, I'm Matt. I'm an entrepreneur who has star...  New York   
4          3  Hi, I'm Matt. I'm an entrepreneur who has star...  New York   

  country      hometown               joined    lat  \
0      us  New York, NY  2007-05-01 22:04:37  40.72   
1      us  New York, NY  2011-01-23 14:13:17  40.72   
2      us  New York, NY  2010-12-30 18:47:34  40.72   
3      us  New York, NY  2011-01-03 14:45:21  40.72   
4      us  New York, NY  2010-12-30 18:34:50  40.72   

                              link   lon  member_name state member_status  \
0  http://www.meetup.com/members/3 -74.0  Matt Meeker    NY        active   
1  http://www.meetup.com/members/3 -74.0  Matt Meeker    NY        active   
2  http://www.meetup.com/members/3 -74.0  Matt Meeker    NY        active   
3  http://www.meetup.com/members/3 -74.0  Matt Meeker    NY        active   
4  http://www.meetup.com/members/3 -74.0  Matt Meeker    NY        active   

               visited  group_id  
0  2009-09-18 18:32:23    490552  
1  2011-03-20 01:02:11   1474611  
2  2011-01-18 20:37:23   1490492  
3  2011-07-23 03:42:28   1515830  
4  2011-06-13 18:33:23   1574965

## Data Cleaning Phase
In this phase, we'll sample a % of the data, and then use "one hot" encoding to turn string features into numbers for our mathematical models.

In [19]:
# I am sampling to 5,000 random samples from the dataframe since it is just SOOO big! This will help my code run faster
df2_sample = df2.sample(n=5000)

In [20]:
#let's explore the df by member id and the first record number of each - what are the features we want to use?
# group a DataFrame (df2_sample) by the 'member_id' column and then select the first row within each group.
# This operation is often used to get the first entry or a representative entry for each unique member ID.
df2_sample.groupby(['member_id']).first()

bio           city  \
member_id                                                                     
3045         It's my favorite element on the periodic table.  San Francisco   
3402                                               not_found       New York   
6948       Coach for Online Dating, and freelance User Ex...  San Francisco   
13149      NY-based angel investor.  Portfolio includes M...       New York   
18613                                      Right up my alley       New York   
...                                                      ...            ...   
48409462                                           not_found       New York   
48417552                                           not_found       New York   
48492382                                           not_found  San Francisco   
48511382                                           not_found  San Francisco   
48530532                                           not_found  San Francisco   

          country           hometown               joined    lat  \
member_id                                                          
3045           us                USA  2012-09-13 20:24:26  37.79   
3402           us                NYC  2016-06-10 20:00:51  40.79   
6948           us       San Clemente  2013-01-07 03:00:45  37.78   
13149          us          not_found  2012-03-18 13:54:48  40.74   
18613          us  Kiev/Philadelphia  2013-08-11 20:35:16  40.74   
...           ...                ...                  ...    ...   
48409462       us     Clearwater, FL  2014-10-18 17:47:50  40.71   
48417552       us           New York  2016-04-19 14:38:21  40.75   
48492382       us          not_found  2015-03-22 06:33:57  37.72   
48511382       us          not_found  2016-08-18 17:59:20  37.80   
48530532       us          not_found  2015-07-10 20:23:16  37.77   

                                             link     lon     member_name  \
member_id                                                                   
3045           http://www.meetup.com/members/3045 -122.41   Neil Mansilla   
3402           http://www.meetup.com/members/3402  -73.95  Chaz Antonelli   
6948           http://www.meetup.com/members/6948 -122.42            Greg   
13149         http://www.meetup.com/members/13149  -74.00    Esther Dyson   
18613         http://www.meetup.com/members/18613  -73.99          Igor S   
...                                           ...     ...             ...   
48409462   http://www.meetup.com/members/48409462  -74.01            Erin   
48417552   http://www.meetup.com/members/48417552  -73.99  Isaiah Pittman   
48492382   http://www.meetup.com/members/48492382 -122.48       @DrJinLee   
48511382   http://www.meetup.com/members/48511382 -122.40           Bella   
48530532   http://www.meetup.com/members/48530532 -122.40        Max Dunn   

          state member_status              visited  group_id  
member_id                                                     
3045         CA        active  2012-09-13 20:24:26   3453602  
3402         NY        active  2017-07-15 14:56:01    107592  
6948         CA        active  2017-10-21 06:22:55   4394122  
13149        NY        active  2015-05-05 00:46:41   1453797  
18613        NY        active  2014-02-21 23:05:40   9716372  
...         ...           ...                  ...       ...  
48409462     NY        active  2015-09-21 18:18:00   1719006  
48417552     NY        active  2016-08-11 00:48:24  10578212  
48492382     CA        active  2015-03-22 06:33:57   1240980  
48511382     CA        active  2016-12-23 04:48:46  20310457  
48530532     CA        active  2015-10-06 20:38:09  18535185  

[4729 rows x 13 columns]

In [21]:
print(df2_sample.columns)


Index(['member_id', 'bio', 'city', 'country', 'hometown', 'joined', 'lat',
       'link', 'lon', 'member_name', 'state', 'member_status', 'visited',
       'group_id'],
      dtype='object')


df2_sample['group_id']: This extracts the 'group_id' column from the DataFrame df2_sample.  
pd.get_dummies(...): This function converts categorical variable(s) into dummy/indicator variables. In this case, it's applied to the 'group_id' column.  
prefix='group_id': This parameter adds a prefix to the newly created dummy variable columns to indicate their association with the original 'group_id'.  
  
  The resulting df2_sample_dummies DataFrame will contain new binary columns (0 or 1) for each unique value in the 'group_id' column. Each row will have a 1 in the column corresponding to its 'group_id' and 0s in the other columns.

In [22]:
#One feature I want to use is the GROUP ID - one thing that we can do is "get dummies" or "one-hot encoding" to
#turn string variables into numbers! look at it below
df2_sample_dummies = pd.get_dummies(df2_sample['group_id'], prefix = 'group_id')

In [23]:
#this is what it looks like to have "dummies" or one-hot encoded variables!
df2_sample_dummies.head()

group_id_6388  group_id_6510  group_id_8458  group_id_8940  \
776591           False          False          False          False   
325767           False          False          False          False   
661615           False          False          False          False   
1409704          False          False          False          False   
293547           False          False          False          False   

         group_id_12907  group_id_14573  group_id_15324  group_id_18843  \
776591            False           False           False           False   
325767            False           False           False           False   
661615            False           False           False           False   
1409704           False           False           False           False   
293547            False           False           False           False   

         group_id_22211  group_id_24765  ...  group_id_26196463  \
776591            False           False  ...              False   
325767            False           False  ...              False   
661615            False           False  ...              False   
1409704           False           False  ...              False   
293547            False           False  ...              False   

         group_id_26241640  group_id_26246864  group_id_26256827  \
776591               False              False              False   
325767               False              False              False   
661615               False              False              False   
1409704              False              False              False   
293547               False              False              False   

         group_id_26264263  group_id_26290335  group_id_26303236  \
776591               False              False              False   
325767               False              False              False   
661615               False              False              False   
1409704              False              False              False   
293547               False              False              False   

         group_id_26336765  group_id_26365189  group_id_26371769  
776591               False              False              False  
325767               False              False              False  
661615               False              False              False  
1409704              False              False              False  
293547               False              False              False  

[5 rows x 2708 columns]

In [24]:
#Let's combine it back to our original dataframe
df2_sample_dummies_concat = pd.concat([df2_sample, df2_sample_dummies], axis=1)

In [25]:
df2_sample_dummies_concat.head()

member_id                                                bio  \
776591    11387669                                          not_found   
325767     6218644                                              Hello   
661615    10238593  I studied French at school, but I need a chanc...   
1409704   33871272                                          not_found   
293547     5588467                                          not_found   

                  city country   hometown               joined    lat  \
776591        New York      us  not_found  2016-07-13 15:15:38  40.75   
325767   San Francisco      us  not_found  2010-03-24 20:50:32  37.79   
661615         Chicago      us  not_found  2009-08-21 23:37:39  41.94   
1409704       New York      us  not_found  2015-05-22 21:24:42  40.74   
293547        New York      us   New York  2008-12-08 02:45:13  40.75   

                                           link     lon       member_name  \
776591   http://www.meetup.com/members/11387669  -73.99  Nitya Narasimhan   
325767    http://www.meetup.com/members/6218644 -122.40       Ahmed Afroz   
661615   http://www.meetup.com/members/10238593  -87.65         Elizabeth   
1409704  http://www.meetup.com/members/33871272  -74.00    Bethany Argiro   
293547    http://www.meetup.com/members/5588467  -73.98            Helene   

         ... group_id_26196463 group_id_26241640 group_id_26246864  \
776591   ...             False             False             False   
325767   ...             False             False             False   
661615   ...             False             False             False   
1409704  ...             False             False             False   
293547   ...             False             False             False   

         group_id_26256827  group_id_26264263  group_id_26290335  \
776591               False              False              False   
325767               False              False              False   
661615               False              False              False   
1409704              False              False              False   
293547               False              False              False   

         group_id_26303236  group_id_26336765  group_id_26365189  \
776591               False              False              False   
325767               False              False              False   
661615               False              False              False   
1409704              False              False              False   
293547               False              False              False   

         group_id_26371769  
776591               False  
325767               False  
661615               False  
1409704              False  
293547               False  

[5 rows x 2722 columns]

In [26]:
#let's repeat the same process for the "cities" feature
df2_sample_dummies_cities = pd.get_dummies(df2_sample_dummies_concat['city'], prefix = 'cities_')
df2_sample_dummies_concat_cities2 = pd.concat([df2_sample_dummies_concat, df2_sample_dummies_cities], axis=1)

In [27]:
df2_sample_dummies_concat_cities2.head()

member_id                                                bio  \
776591    11387669                                          not_found   
325767     6218644                                              Hello   
661615    10238593  I studied French at school, but I need a chanc...   
1409704   33871272                                          not_found   
293547     5588467                                          not_found   

                  city country   hometown               joined    lat  \
776591        New York      us  not_found  2016-07-13 15:15:38  40.75   
325767   San Francisco      us  not_found  2010-03-24 20:50:32  37.79   
661615         Chicago      us  not_found  2009-08-21 23:37:39  41.94   
1409704       New York      us  not_found  2015-05-22 21:24:42  40.74   
293547        New York      us   New York  2008-12-08 02:45:13  40.75   

                                           link     lon       member_name  \
776591   http://www.meetup.com/members/11387669  -73.99  Nitya Narasimhan   
325767    http://www.meetup.com/members/6218644 -122.40       Ahmed Afroz   
661615   http://www.meetup.com/members/10238593  -87.65         Elizabeth   
1409704  http://www.meetup.com/members/33871272  -74.00    Bethany Argiro   
293547    http://www.meetup.com/members/5588467  -73.98            Helene   

         ... group_id_26365189 group_id_26371769 cities__Chicago  \
776591   ...             False             False           False   
325767   ...             False             False           False   
661615   ...             False             False            True   
1409704  ...             False             False           False   
293547   ...             False             False           False   

         cities__Chicago Heights  cities__Chicago Ridge  cities__New York  \
776591                     False                  False              True   
325767                     False                  False             False   
661615                     False                  False             False   
1409704                    False                  False              True   
293547                     False                  False              True   

         cities__San Francisco  cities__South San Francisco  \
776591                   False                        False   
325767                    True                        False   
661615                   False                        False   
1409704                  False                        False   
293547                   False                        False   

         cities__West Chicago  cities__West New York  
776591                  False                  False  
325767                  False                  False  
661615                  False                  False  
1409704                 False                  False  
293547                  False                  False  

[5 rows x 2730 columns]

In [28]:
#write it to a CSV before your kernel dies! this could be helpful if you want to use the same sample again in the future
df2_sample_dummies_concat_cities2.to_csv('members2.csv')

In [19]:
!cp members2.csv "gdrive/My Drive/Work/UBB 2023/02_Cursuri/Session #5/data/"

## Training the model
We will use groupIDs and Cities as our main features to train the model - and since we want the data itself to group and tell us what the major groups are - we are going to use a technique called "clustering" (or k-means clustering, where k= the number of clusters). I don't know in advance what the best number of clusters will be, so I will try a bunch of different k (k= 2,4,6,8) and then see how well our clusters are performing. See below :)

In [29]:
df2_sample_dummies_concat_cities2_train = df2_sample_dummies_concat_cities2.loc[:, 'group_id_6388':'cities__West New York']
df2_sample_dummies_concat_cities2_train.head()

group_id_6388  group_id_6510  group_id_8458  group_id_8940  \
776591           False          False          False          False   
325767           False          False          False          False   
661615           False          False          False          False   
1409704          False          False          False          False   
293547           False          False          False          False   

         group_id_12907  group_id_14573  group_id_15324  group_id_18843  \
776591            False           False           False           False   
325767            False           False           False           False   
661615            False           False           False           False   
1409704           False           False           False           False   
293547            False           False           False           False   

         group_id_22211  group_id_24765  ...  group_id_26365189  \
776591            False           False  ...              False   
325767            False           False  ...              False   
661615            False           False  ...              False   
1409704           False           False  ...              False   
293547            False           False  ...              False   

         group_id_26371769  cities__Chicago  cities__Chicago Heights  \
776591               False            False                    False   
325767               False            False                    False   
661615               False             True                    False   
1409704              False            False                    False   
293547               False            False                    False   

         cities__Chicago Ridge  cities__New York  cities__San Francisco  \
776591                   False              True                  False   
325767                   False             False                   True   
661615                   False             False                  False   
1409704                  False              True                  False   
293547                   False              True                  False   

         cities__South San Francisco  cities__West Chicago  \
776591                         False                 False   
325767                         False                 False   
661615                         False                 False   
1409704                        False                 False   
293547                         False                 False   

         cities__West New York  
776591                   False  
325767                   False  
661615                   False  
1409704                  False  
293547                   False  

[5 rows x 2716 columns]

In [30]:
#k = 8 training model
km = KMeans(n_clusters=8)
%time km.fit(df2_sample_dummies_concat_cities2_train)
clusters = km.labels_.tolist()
%time km.fit(df2_sample_dummies_concat_cities2_train)
silhouette_k8 = silhouette_score(df2_sample_dummies_concat_cities2_train[0:5000], clusters[0:5000])

CPU times: total: 594 ms
Wall time: 1.21 s
CPU times: total: 672 ms
Wall time: 648 ms


In [31]:
#k=6 training model
km = KMeans(n_clusters=6)
%time km.fit(df2_sample_dummies_concat_cities2_train)
clusters = km.labels_.tolist()
%time km.fit(df2_sample_dummies_concat_cities2_train)
silhouette_k6 = silhouette_score(df2_sample_dummies_concat_cities2_train[0:5000], clusters[0:5000])

CPU times: total: 312 ms
Wall time: 485 ms
CPU times: total: 438 ms
Wall time: 577 ms


In [32]:
# k = 4 training model
km = KMeans(n_clusters=4)
%time km.fit(df2_sample_dummies_concat_cities2_train)
clusters = km.labels_.tolist()
%time km.fit(df2_sample_dummies_concat_cities2_train)
silhouette_k4 = silhouette_score(df2_sample_dummies_concat_cities2_train[0:5000], clusters[0:5000])

CPU times: total: 219 ms
Wall time: 446 ms
CPU times: total: 172 ms
Wall time: 455 ms


In [33]:
# k = 2 training model
km = KMeans(n_clusters=2)
%time km.fit(df2_sample_dummies_concat_cities2_train)
clusters = km.labels_.tolist()
%time km.fit(df2_sample_dummies_concat_cities2_train)
silhouette_k2 = silhouette_score(df2_sample_dummies_concat_cities2_train[0:5000], clusters[0:5000])

CPU times: total: 266 ms
Wall time: 407 ms
CPU times: total: 234 ms
Wall time: 428 ms


## Model Evaluation


In [34]:
silhouette = [silhouette_k2, silhouette_k4, silhouette_k6, silhouette_k8]
count_k = [2, 4, 6, 8]

count_silhouette = list(zip(count_k, silhouette))
print(count_silhouette)

[(2, np.float64(0.23375482624473376)), (4, np.float64(0.2127501585142326)), (6, np.float64(0.12711961903971894)), (8, np.float64(0.050277763758758066))]


In [35]:
plt.plot(*zip(*count_silhouette))

* No Clear Elbow:
If the plot doesn't show a clear elbow and the inertia decreases smoothly without a significant break point, it might be challenging to determine the optimal k based solely on this method.  
  
* Slight Elbow:
If there's a slight elbow, it suggests that adding more clusters beyond that point provides diminishing returns in terms of explaining the variance in the data.  
  
* Sharp Elbow:
If there's a sharp elbow, it indicates a more pronounced point where the addition of clusters is less beneficial. This is often considered a good candidate for the optimal number of clusters.  

  However, it's important to note that the elbow method has limitations, and the choice of k may not always be entirely clear-cut. It's often used as a heuristic rather than a strict rule. Other factors, such as domain knowledge, the context of the problem, and the specific characteristics of the data, should also be considered.

## Model Output
Now that we've decided on k=4 clusters, let's assign the cluster labels back to the original data, and make it interpretable!

In [36]:
km = KMeans(n_clusters=4)
%time km.fit(df2_sample_dummies_concat_cities2_train)
clusters4 = km.labels_.tolist()
silhouette_k4 = silhouette_score(df2_sample_dummies_concat_cities2_train[0:50000], clusters[0:50000])

CPU times: total: 375 ms
Wall time: 466 ms


In [37]:
df2_sample_dummies_concat_cities2_train.loc[:, "cluster_number"] = clusters4

In [38]:
df2_sample_dummies_concat_cities2_train.head()

group_id_6388  group_id_6510  group_id_8458  group_id_8940  \
776591           False          False          False          False   
325767           False          False          False          False   
661615           False          False          False          False   
1409704          False          False          False          False   
293547           False          False          False          False   

         group_id_12907  group_id_14573  group_id_15324  group_id_18843  \
776591            False           False           False           False   
325767            False           False           False           False   
661615            False           False           False           False   
1409704           False           False           False           False   
293547            False           False           False           False   

         group_id_22211  group_id_24765  ...  group_id_26371769  \
776591            False           False  ...              False   
325767            False           False  ...              False   
661615            False           False  ...              False   
1409704           False           False  ...              False   
293547            False           False  ...              False   

         cities__Chicago  cities__Chicago Heights  cities__Chicago Ridge  \
776591             False                    False                  False   
325767             False                    False                  False   
661615              True                    False                  False   
1409704            False                    False                  False   
293547             False                    False                  False   

         cities__New York  cities__San Francisco  cities__South San Francisco  \
776591               True                  False                        False   
325767              False                   True                        False   
661615              False                  False                        False   
1409704              True                  False                        False   
293547               True                  False                        False   

         cities__West Chicago  cities__West New York  cluster_number  
776591                  False                  False               0  
325767                  False                  False               1  
661615                  False                  False               2  
1409704                 False                  False               0  
293547                  False                  False               0  

[5 rows x 2717 columns]

In [39]:
df2_sample_dummies_concat_cities2_train["cluster_number"].value_counts()

cluster_number
0    2843
1    1353
2     799
3       5
Name: count, dtype: int64

## Tadah! We have a merged dataframe of members, clustered by their city and groups they're interested in, merged on what the group categories are (from the original groups df) and we can now export this and explore!

In [40]:
df2_sample.head()

member_id                                                bio  \
776591    11387669                                          not_found   
325767     6218644                                              Hello   
661615    10238593  I studied French at school, but I need a chanc...   
1409704   33871272                                          not_found   
293547     5588467                                          not_found   

                  city country   hometown               joined    lat  \
776591        New York      us  not_found  2016-07-13 15:15:38  40.75   
325767   San Francisco      us  not_found  2010-03-24 20:50:32  37.79   
661615         Chicago      us  not_found  2009-08-21 23:37:39  41.94   
1409704       New York      us  not_found  2015-05-22 21:24:42  40.74   
293547        New York      us   New York  2008-12-08 02:45:13  40.75   

                                           link     lon       member_name  \
776591   http://www.meetup.com/members/11387669  -73.99  Nitya Narasimhan   
325767    http://www.meetup.com/members/6218644 -122.40       Ahmed Afroz   
661615   http://www.meetup.com/members/10238593  -87.65         Elizabeth   
1409704  http://www.meetup.com/members/33871272  -74.00    Bethany Argiro   
293547    http://www.meetup.com/members/5588467  -73.98            Helene   

        state member_status              visited  group_id  
776591     NY        active  2016-10-27 02:57:01  20167049  
325767     CA        active  2013-01-26 02:56:40   1500013  
661615     IL        active  2009-08-26 03:01:29     63782  
1409704    NY        active  2016-03-23 16:25:27  18366468  
293547     NY        active  2008-12-08 15:09:41    233753

In [41]:
df2_sample.loc[:, "clusters_number"] = clusters4

In [42]:
df2_sample.to_csv('members_cluster_group.csv')

In [43]:
!cp members_cluster_group.csv "gdrive/My Drive/Work/UBB 2023/02_Cursuri/Session #5/data/"

'cp' is not recognized as an internal or external command,
operable program or batch file.


In [44]:
df2_sample.head()

member_id                                                bio  \
776591    11387669                                          not_found   
325767     6218644                                              Hello   
661615    10238593  I studied French at school, but I need a chanc...   
1409704   33871272                                          not_found   
293547     5588467                                          not_found   

                  city country   hometown               joined    lat  \
776591        New York      us  not_found  2016-07-13 15:15:38  40.75   
325767   San Francisco      us  not_found  2010-03-24 20:50:32  37.79   
661615         Chicago      us  not_found  2009-08-21 23:37:39  41.94   
1409704       New York      us  not_found  2015-05-22 21:24:42  40.74   
293547        New York      us   New York  2008-12-08 02:45:13  40.75   

                                           link     lon       member_name  \
776591   http://www.meetup.com/members/11387669  -73.99  Nitya Narasimhan   
325767    http://www.meetup.com/members/6218644 -122.40       Ahmed Afroz   
661615   http://www.meetup.com/members/10238593  -87.65         Elizabeth   
1409704  http://www.meetup.com/members/33871272  -74.00    Bethany Argiro   
293547    http://www.meetup.com/members/5588467  -73.98            Helene   

        state member_status              visited  group_id  clusters_number  
776591     NY        active  2016-10-27 02:57:01  20167049                0  
325767     CA        active  2013-01-26 02:56:40   1500013                1  
661615     IL        active  2009-08-26 03:01:29     63782                2  
1409704    NY        active  2016-03-23 16:25:27  18366468                0  
293547     NY        active  2008-12-08 15:09:41    233753                0

In [45]:
df2_sample['city'].nunique()

8

In [46]:
df2_sample['group_id'].nunique()

2708